# GKR By Hand

## Introduction

The Goldwasser-Kalai-Rothblum (GKR) protocol is an advanced technique in the field of verifiable computation. Before diving into the details, let's understand the big picture:

- Purpose: GKR allows a prover to convince a verifier that a computation was performed correctly, without the verifier having to redo the entire computation.
- Key Concept: The computation is represented as an arithmetic circuit, which is then verified layer by layer.
- Main Components:

  - Prover: Performs the computation and generates proofs
  - Verifier: Checks the proofs without redoing the full computation
  - Sumcheck Protocol: A core subroutine used in GKR


- Process Overview:
  1. Convert the computation to an arithmetic circuit
  2. Verify the circuit layer by layer, starting from the output
  3. Use interactive proofs between the prover and verifier
  4. Apply optimization techniques to improve efficiency

- What This Document Covers:

  - Detailed explanation of the GKR protocol steps
  - A hands-on example with a simple arithmetic circuit
  - Implementation of key protocol components in Python
  - Optimization techniques, including the use of lookup tables


By the end of this document, you should understand how GKR works at a fundamental level and be able to follow a basic implementation of the protocol.

## Protocol Framework

The GKR protocol is based on the idea of verifying computation by checking the correctness of each step through a series of interactive proofs between a prover (who performs the computation) and a verifier (who checks the computation). The protocol uses a layered approach, where each layer of the computation is verified independently.

### Key Components

- **Circuit Representation**: The computation is represented as an arithmetic circuit. An arithmetic circuit is like a recipe for a calculation. Imagine a flowchart where each step is either adding or multiplying numbers. The numbers flow from the bottom (inputs) to the top (output), getting combined along the way.
- **Multilinear Extension (MLE)**: An MLE is a way to stretch a function that only works on 0s and 1s so that it can handle any number. It's like turning a light switch (on/off) into a dimmer switch (any brightness level). 
- **Sumcheck Protocol**: A key component used in the GKR protocol to verify sums of polynomials efficiently. The sumcheck protocol is a way to prove that a big sum is correct without actually adding up all the numbers. It's like proving you counted all the jellybeans in a jar without having to count them all again. This is the core technique that makes GKR efficient. It allows the verifier to check sums quickly, which is essential for verifying each layer of the circuit.
- **Interactive Proof**: Interactions between a prover and a verifier, where the prover demonstrates the correctness of their computation, and the verifier confirms this correctness through challenges and checks.

## Detailed Steps

1. **Circuit Setup**: The computation to be verified is structured as an arithmetic circuit with a hierarchical layout.
2. **Initialization**: The prover and verifier establish the initial parameters and inputs for the computation.
3. **Layer by Layer Verification**:
   - The verifier sends a random challenge related to the output of a specific layer.
   - The prover responds with a proof showing the correctness of the output for that layer.
   - The verifier checks the proof and proceeds to the next layer.
4. **Final Check**: Upon reaching the input layer, a final verification is performed to ensure all layers are computed correctly.

## Example

![Circuit Diagram](ac.png)

### Legend
- Circles represent Add (Addition, light yellow) or Mult (Multiplication, purple) gates.
- Each node in the circuit is labeled as `V_i(x) = value`, where `i` denotes the layer, `x` denotes the input bits, and `value` denotes the computed value.

### Step 1: Protocol Initialization
The protocol starts from the root node of the output layer. The verifier (V) wants to verify the correctness of the computation performed by the prover (P).

#### Input and Output Layers
- **Input Layer:** Nodes at the bottom (e.g., `V_3(0,0) = 1`, `V_3(1,1) = 4`, etc.)
- **Output Layer:** Nodes at the top (e.g., `V_0(0) = 4`, `V_0(1) = 2`, etc.)

### Step 2: Sumcheck Protocol

The following computation follows the *Protocol 2 of Libra*, contains no optimization of bookkeeping table, which results the time complexicity to be $O(n \cdot log(n))$. And for the simiplicity of computation, the following computation does not involve module operation, meaning it is not done on field.

#### Round 1: Initialization

For the output level, we have the output layer multilinear equation (MLE): 

$$
\begin{aligned}
V_0(x_1) =  & (1-x_1) \times 2 + (x_1) \times 2
\end{aligned}
$$

The V will choose a random number (challenge) $g^{(0)} = (2)$. Both P and V will compute $V_0(g^{(0)})$, as 

$$
\begin{aligned}
V_0(2) =  (1-2) \times 2 + (2) \times 2 = 3 + 4 = 2
\end{aligned}
$$

In [51]:
import numpy as np
import random

# All of our computation will be done in F_5
# In real case, 5 will be too small to be practical
modulus = 5

# We fix the seed to make program execution to be consistent with text content
random.seed(77777777)

# This function generate MLE for given output values
def generate_multilinear_extension(values):
    n = int(np.log2(len(values)))
    indices = [(i >> k) & 1 for i in range(len(values)) for k in range(n)]
    table = {tuple(indices[i*n:(i+1)*n]): values[i] for i in range(len(values))}
    def multilinear_extension(input_vars):
        input_vars = tuple(input_vars)
        if input_vars in table:
            return table[input_vars]
        # Calculate multilinear extension for input_vars
        result = 0
        for key, value in table.items():
            prod = value
            for i in range(n):
                if key[i] == 1:
                    prod *= input_vars[i]
                else:
                    prod *= (1 - input_vars[i])
            result += prod
        return result % modulus
    return multilinear_extension

# Generate the MLE for output level
V0_MLE   = generate_multilinear_extension([2, 2]) # Prover need to compute this from processing the circuit
g_0      = random.randrange(modulus)
V_g      = V0_MLE([g_0])
print(f"The proof target is g_0 = {g_0}, V_0(g_0) = {V_g}")

The proof target is g_0 = 2, V_0(g_0) = 2


#### Round 2: First Sumcheck Protocol

$$
\tilde{V}_0(g^{(0)}) = \tilde{V}_0(2) = \sum_{x,y \in \{0,1\}^{s_1}} \tilde{mult}_1(g^{(0)}, x, y) (\tilde{V}_1(x) \tilde{V}_1(y)) + \tilde{add}_1(g^{(0)}, x, y) (\tilde{V}_1(x) + \tilde{V}_1(y)) = 2
$$

So the next step here in the round 2 is that we need to use Sumcheck to prove the above result is equal to $\tilde{V}_0(g^{(0)}) = 2$.

First of all, Prover needs to calculate what is exactly $\tilde{m}ult_1(g^{(0)}, x_1, x_2, y_1, y_2)$ and $\tilde{a}dd_1(g^{(0)}, x_1, x_2, y_1, y_2)$.

We know that by the definition of wiring predicates. There two *mult* gates on layer 0. The first *mult* gate is $V_0(0)$, which connects $V_1(00)$ and $V_1(10)$.

So by definition of MLE, we can know that

$$
\begin{aligned}
&\tilde{mult}_1(g_1, x_1, x_2, y_1, y_2)\\
=& (1-g_1)(1-x_1)(1-x_2)(y_1)(1-y_2) \\
=& 4(1-x_1)(1-x_2)(y_1)(1-y_2)
\end{aligned}
$$

In [52]:
# Prover can know from the circuit that: between level 0 and level 1
# there is only one mult gate, connect V0(0), V1(00), V1(10)
# We convert bit string [0, 00, 10] into number, we got 5b'01000 = 8
# So for mult predicate, we have 
mult1_tilde_values = [0] * 32
mult1_tilde_values[8] = 1
mult1_tilde_mle_raw = generate_multilinear_extension(mult1_tilde_values)

In [53]:
# Now, because we already have the random g_1 = 2, we need to reduce such MLE to be 4 variables
# we can further define such function to make such reduce work
def reduce_variables(mle_func, fixed_values):
    def reduced_func(input_vars):
        full_vars = fixed_values[:]
        j = 0
        for i in range(len(fixed_values)):
            # If input is `str`, it means non-fixed variables
            if isinstance(fixed_values[i], str):
                full_vars[i] = input_vars[j]
                j += 1
        return mle_func(full_vars)
    return reduced_func

# Know we can get the actual MLE for the mult. wiring predicate for level 0~1
mult1_tilde = reduce_variables(mult1_tilde_mle_raw, [g_0, 'x1', 'x2', 'y1', 'y2'])
print(f"mult1_tilde([0,0,1,0]) = {mult1_tilde([0,0,1,0])}")

mult1_tilde([0,0,1,0]) = 4


Similarly, we can also know the MLE for *add* wiring predicate is

$$
\begin{aligned}
&\tilde{add}_1(g_1, x_1, x_2, y_1, y_2)\\
=& (g_1)(1-x_1)(x_2)(y_1)(y_2) \\
=& 2(1-x_1)(x_2)(y_1)(y_2)
\end{aligned}
$$

In [54]:
# We can do the same trick here. Prover will need to examine the circuit and generate
# MLE for addition predicate.
# Because there is only one Addition gate and the bit string is [1, 01, 11]
# it means that 5b'11101 -> 29 in decimal
add_tilde_values = [0] * 32
add_tilde_values[29] = 1
add_tilde_mle_raw = generate_multilinear_extension(add_tilde_values)
add1_tilde = reduce_variables(add_tilde_mle_raw, [g_0, 'x1', 'x2', 'y1', 'y2'])
print(f"add1_tilde([0, 1, 1, 1]) = {add1_tilde([0, 1, 1, 1])}")

add1_tilde([0, 1, 1, 1]) = 2


We can replace the $g^{(0)}$ with $2$ on the equation above, so we have the proof target to generate.

Please note here that $s_1 = 2$ because there are 4 gates on the second layer.

For the simplicity of computation, we use the following equations:

$$
\begin{aligned}
   g_{mult}(x_1,x_2,y_1,y_2) = 4(1-x_1)(1-x_2)(y_1)(1-y_2) \tilde{V}_1(x_1, x_2) \tilde{V}_1(y_1, y_2)
\end{aligned}
$$

and

$$
\begin{aligned}
   g_{add}(x_1,x_2,y_1,y_2) = 2(1-x_1)(x_2)(y_1)(y_2)[\tilde{V}_1(x_1, x_2) + \tilde{V}_1(y_1, y_2)]
\end{aligned}
$$

So now, our proving target is

$$
\begin{aligned}
2 =& \sum_{x_1,x_2,y_1,y_2 \in \{0,1\}} g_{add}(x_1,x_2,y_1,y_2) + g_{mult}(x_1, x_2, y_1, y_2)\\
=& \sum_{x_1,x_2,y_1,y_2 \in \{0,1\}}g(x_1,x_2,y_1,y_2)
\end{aligned}
$$

where $g(x_1,x_2,y_1,y_2) = g_{add}(x_1,x_2,y_1,y_2) + g_{mult}(x_1,x_2,y_1,y_2)$

We also need to know the MLE to represent the gate output $V_1(b_1, b_2)$ as shown below:

$$
V_1(b_1, b_2) = 2(1-b_1)(1-b_2) + 4(1-b_1)b_2 + b_1(1-b_2) + 3b_1b_2
$$

In [55]:
# Prover need to do the same thing for V1's MLE, by examining the values on level 1, we have
V1_MLE = generate_multilinear_extension([2, 1, 4, 3])
print(f"V1_MLE(0, 0) = {V1_MLE([0, 0])})")
print(f"V1_MLE(0, 1) = {V1_MLE([0, 1])})")
print(f"V1_MLE(1, 0) = {V1_MLE([1, 0])})")
print(f"V1_MLE(1, 1) = {V1_MLE([1, 1])})")

V1_MLE(0, 0) = 2)
V1_MLE(0, 1) = 4)
V1_MLE(1, 0) = 1)
V1_MLE(1, 1) = 3)


In order to proceed with the sumcheck proof, *Prover* needs to send the *Verifier* with an MLE $H_1(x_1)$ and claims that:

$$
H_1(x_1) = \sum_{x_2,y_1,y_2 \in \{0,1\}}g(x_1,x_2,y_1,y_2)
$$

Please note here that $x_1$ has been moved out of summation range. So, if the proof holds, we must have:

$$
\tilde{V}_0(g^{(0)}) = 2 = H_1(0) + H_1(1)
$$

The *Prover* will send $H_1(x_1)$ to *Verifier*, *Verifier* will check if $2 = H_1(0) + H_1(1)$ holds or not. If the equation holds, then we proceed to next round of sumcheck. If not, *Verifier* will reject and stop.

So the question is how should *Prover* calculate the $H_1(x_1)$?

The simple answer is: we just do it via *BruteForce*. In this tutorial, we will do this for one time and the rest of calculations are just the same

We can expand the expression of $H_1(x_1)$

$$
\begin{aligned}
H_1(x_1) &= \tilde{mult}_1(2, x_1, 0, 1, 0) \tilde{V}_1(x_1, 0) \tilde{V}_1(1, 0) + \tilde{add}_1(2, x_1,1,1,1)(\tilde{V}_1(x_1, 1)+\tilde{V}_1(1, 1))\\
&= 4(1-x_1)(2(1-x_1)+x_1) + 2(1-x_1)(4(1-x_1)+3x_1+3) \\
&= (1-x_1)(2-x_1)
\end{aligned}
$$

The $\tilde{V}_0(g)$ function is a crucial component of our GKR implementation. It represents the computation we're trying to verify at the output layer of our arithmetic circuit. Here's what it does:

1. It takes input variables x1, x2, y1, y2 which represent the "wires" in our circuit.
2. It combines the multiplication and addition terms based on our circuit structure.
3. The function returns the result modulo our chosen field size.

This function will be used repeatedly in our sumcheck protocol to evaluate different points in our extended circuit.

In [56]:
# By combining everything together, we have our proving target:
def V0_g(input_vars):
    x1, x2, y1, y2 = input_vars
    mult_term = mult1_tilde([x1, x2, y1, y2]) * V1_MLE([x1, x2]) * V1_MLE([y1, y2])
    add_term  = add1_tilde([x1, x2, y1, y2]) * (V1_MLE([x1, x2]) + V1_MLE([y1, y2]))
    print(f"x1 = {x1}, x2 = {x2}, y1 = {y1}, y2 = {y2},  mult_term = {mult_term}, add_term = {add_term}")
    return (mult_term + add_term) % modulus

# The first round of sumcheck, prover need to compute a H1(x1)
# It basically traverse the combination of all other three variables (x2, y1, y2)
def H1_x1(x1):
    return (V0_g([x1, 0, 0 ,0]) + V0_g([x1, 1, 0 ,0]) + V0_g([x1, 0, 1, 0]) + V0_g([x1, 1, 1, 0]) + V0_g([x1, 0, 0 ,1]) + V0_g([x1, 1, 0 ,1]) + V0_g([x1, 0, 1, 1]) + V0_g([x1, 1, 1, 1])) % modulus

# Prover will seed this H1_x1 to verifier and evaluate on 0 and 1
# and check if the sum is equal to V0(g0)
print(f"Verifier; H1_x1(0) = {H1_x1(0)}, H1(1) = {H1_x1(1)}, Pass = {(H1_x1(0) + H1_x1(1)) == V_g}")

x1 = 0, x2 = 0, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 0,  mult_term = 8, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 1,  mult_term = 0, add_term = 14
x1 = 1, x2 = 0, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 1, x2 = 1, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 1, x2 = 0, y1 = 1, y2 = 0,  mult_term = 0, add_term = 0
x1 = 1, x2 = 1, y1 = 1, y2 = 0,  mult_term = 0, add_term = 0
x1 = 1, x2 = 0, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 1, x2 = 1, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 1, x2 = 0, y1 = 1, y2 = 1,  mult_term = 0, add_term = 0
x1 = 1, x2 = 1, y1 = 1, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 0,

Before we can proceed to the next round, *Verifier* will pick a random number, let say $x_1 = 3$.  *Verifier* will return $H_1(3)=2$ back to *Prover*, which becomes the proving target for next round.

Similarly, as the second round of sumcheck protocol, we need to prove that $H_1(3) = H_2(0) + H_2(1)$, where

$$
2 = H_2(x_2) = \sum_{y_1,y_2 \in \{0,1\}}g(3,x_2,y_1,y_2)
$$

In [57]:
random_x1 = random.randrange(modulus)
print(f"Verifier returns random_x1 = {random_x1}, H1_x1(random_x1) = {H1_x1(random_x1)})")

x1 = 0, x2 = 0, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 0,  mult_term = 8, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 1,  mult_term = 0, add_term = 14
Verifier returns random_x1 = 0, H1_x1(random_x1) = 2)


The $H_1(x_1)$ function implements the first round of our sumcheck protocol. Here's what's happening:

1. It sums up all possible combinations of the remaining variables (x2, y1, y2).
2. This sum represents the claim we're making about the output layer of our circuit.
3. By evaluating this function at 0 and 1, we're creating the polynomial for the first round of sumcheck.

The verifier will check if the sum of these evaluations matches our initial claim about the circuit output.


Now we're moving to the second round of sumcheck. The $H_2(x_2)$ function is similar to $H_1(x_1)$, but with a key difference:

1. We're now using the random challenge random_x1 chosen by the verifier in the previous round.
2. We're summing over fewer variables (just y1 and y2) because x1 has been fixed.

This process of fixing variables and summing over the remaining ones continues for each round of the sumcheck protocol.

Similarly, we can expand the expression above

$$
\begin{aligned}
H_2(x_2) &= \tilde{m}ult_1(2, 3, x_2, 1, 0) \tilde{V}_1(3, x_2) \tilde{V}_1(1, 0) + \tilde{a}dd_1(2, 3,x_2,1,1)(\tilde{V}_1(3, x_2)+\tilde{V}_1(1, 1))\\
&= 3(x_2-1)(2x_2-1) + x_2(2x_2+2)
\end{aligned}
$$

In [58]:
def H2_x2(x2):
    return (V0_g([random_x1, x2, 0 ,0]) + V0_g([random_x1, x2, 1, 0]) + V0_g([random_x1, x2, 0 ,1]) + V0_g([random_x1, x2, 1, 1])) % modulus

*Prover* sends the $H_2(x_2) = 3(x_2-1)(2x_2-1) + x_2(2x_2+2)$, *Verifier* evaluates $H_2(0)$ and $H_2(1)$

$$
\begin{aligned}
H_2(0) = 3(-1)(-1)+0=3\\
H_2(1) = 1(2+2) = 4
\end{aligned}
$$

In [59]:
H2_0 = H2_x2(0)
H2_1 = H2_x2(1)
print(f"Verifier; H2_x2(0) = {H2_0}, H2_x2(1) = {H2_1}, Pass = {(H2_0 + H2_1)%modulus == H1_x1(random_x1)}")

x1 = 0, x2 = 0, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 0,  mult_term = 8, add_term = 0
x1 = 0, x2 = 0, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 1,  mult_term = 0, add_term = 14
x1 = 0, x2 = 0, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 0,  mult_term = 8, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 0, y1 = 1, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 1, y1 = 1, y2 = 1,  mult_term = 0, add_term = 14
Verifier; H2_x2(0) = 3

So we have$H_2(0)+H_2(1) = 3 + 4\ mod\ 5 = 2$, which is equal to $H_1(3) = 2$, so *Verifier* accepts and picks another random number as $x_2$, let say $x_2 = 4$, and the sumcheck protocol proceeds to the next round.

In [60]:
# We can continue with this computation to get all four random numbers.
random_x2 = random.randrange(modulus)
print(f"Verifier returns random_x2 = {random_x2}, H2_x2(random_x2) = {H2_x2(random_x2)})")

def H3_y1(y1):
    return (V0_g([random_x1, random_x2, y1 ,0]) + V0_g([random_x1, random_x2, y1, 1])) % modulus

H3_0 = H3_y1(0)
H3_1 = H3_y1(1)
print(f"Verifier; H3_y1(0) = {H3_0}, H3_y1(1) = {H3_1}, Pass = {(H3_0 + H3_1)%modulus == H2_x2(random_x2)}")

random_y1 = random.randrange(modulus)
print(f"Verifier returns random_y1 = {random_y1}, H3_y1(random_y1) = {H3_y1(random_y1)})")

def H4_y2(y2):
    return (V0_g([random_x1, random_x2, random_y1 ,y2])) % modulus
H4_0 = H4_y2(0)
H4_1 = H4_y2(1)
print(f"Verifier; H4_y2(0) = {H4_0}, H4_y2(1) = {H4_1}, Pass = {(H4_0 + H4_1)%modulus == H3_y1(random_y1)}")

random_y2 = random.randrange(modulus)
print(f"Verifier returns random_y2 = {random_y2}, H4_y2(random_y2) = {H4_y2(random_y2)})")

x1 = 0, x2 = 2, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 0,  mult_term = 1, add_term = 0
x1 = 0, x2 = 2, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 1,  mult_term = 0, add_term = 16
Verifier returns random_x2 = 2, H2_x2(random_x2) = 2)
x1 = 0, x2 = 2, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 2, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 0,  mult_term = 1, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 1,  mult_term = 0, add_term = 16
x1 = 0, x2 = 2, y1 = 0, y2 = 0,  mult_term = 0, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 0,  mult_term = 1, add_term = 0
x1 = 0, x2 = 2, y1 = 0, y2 = 1,  mult_term = 0, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 1,  mult_term = 0, add_term = 16
Verifier; H3_y1(0) = 0, H3_y1(1) = 2, Pass = True
x1 = 0, x2 = 2, y1 = 1, y2 = 0,  mult_term = 1, add_term = 0
x1 = 0, x2 = 2, y1 = 1, y2 = 1,  mult_term = 0, add_term = 16
Verifier returns random_y1 = 1, H3_y1(

#### Last Round: Random Combination

At the end of this layer's sumcheck, *Verifier* have four random number: $(x_1,x_2,y_1,y_2) = (3,4,1,2)$. *Verifier* will check if:

$$
H_4(y_2) = H_4(2) == \tilde{m}ult_1(2, 3, 4, 1, 2) \tilde{V}_1(3, 4) \tilde{V}_1(1, 2) + \tilde{a}dd_1(2,3,4,1,2)(\tilde{V}_1(3, 4)+\tilde{V}_1(1, 2))
$$

The computation is the same as previous ones, assuming the last round of sumcheck for Layer 1 pass. We will next proceed to the Layer 2 of GKR and proving target becomes the random combination of $\tilde{V}_1(3, 4)$ and $\tilde{V}_1(1, 2)$. We pick two random number from the field, let say $(\alpha, \beta) = (3, 1)$, So our proving target becomes:

$$
\alpha \tilde{V}_1(3, 4) + \beta \tilde{V}_1(1, 2) = 3(2(1−3)(1−4) + 4(1−3)(4) + 3(1−4) + 3(3)(4)) + 2(0) = 1
$$

So the proving target for the Layer 2 of GKR is

$$
1 = \sum_{x,y \in \{0,1\}^{s_i+1}} \left( (\alpha^{(i)} \tilde{mult}_{i+1}(u^{(i)}, x, y) + \beta^{(i)} \tilde{mult}_{i+1}(v^{(i)}, x, y)) (\tilde{V}_{i+1}(x) \tilde{V}_{i+1}(y)) \right) + \left( (\alpha^{(i)} \tilde{add}_{i+1}(u^{(i)}, x, y) + \beta^{(i)} \tilde{add}_{i+1}(v^{(i)}, x, y)) (\tilde{V}_{i+1}(x) + \tilde{V}_{i+1}(y)) \right)
$$

We can notice that Layer 1 only contains $mult$ gates, for the simplicity of illustration, our proving target becomes

$$
1 = \sum_{x,y \in \{0,1\}^{s_i+1}} (\alpha^{(i)} \tilde{mult}_{i+1}(u^{(i)}, x, y) + \beta^{(i)} \tilde{mult}_{i+1}(v^{(i)}, x, y)) (\tilde{V}_{i+1}(x) \tilde{V}_{i+1}(y))
$$

By adding the random challenges $(\alpha, \beta) = (3, 1)$ and $(x_1,x_2,y_1,y_2) = (3,4,1,2)$ from previous layer, we have

$$
1 = \sum_{x,y \in \{0,1\}^{s_i+1}} (3 \times \tilde{mult}_{i+1}((3,4), x, y) + 1 \times \tilde{mult}_{i+1}((1,2), x, y)) (\tilde{V}_{i+1}(x) \tilde{V}_{i+1}(y))
$$

In [61]:
alpha = random.randrange(modulus)
beta = random.randrange(modulus)

ProveTarget = (alpha * V1_MLE([random_x1, random_x2]) + beta * V1_MLE([random_y1, random_y2])) % modulus
print(f"ProveTarget = {ProveTarget}, (random_x1, random_x2, random_y1, random_y2) = {[random_x1, random_x2, random_y1, random_y2]}")

ProveTarget = 4, (random_x1, random_x2, random_y1, random_y2) = [0, 2, 1, 2]


### Step 3: Linear GKR

We're now transitioning to an optimized version of GKR called Linear GKR. The key idea here is to use lookup tables to speed up our computations. Here's what to expect:

1. We'll create functions to pre-compute certain values and store them in lookup tables.
2. This allows us to replace expensive real-time calculations with faster table lookups.
3. The overall structure of the protocol remains the same, but the individual steps become more efficient.

In order to prove the equation above, we rewrite the equation above as

$$
\sum_{x_1,x_2,y_1,y_2 \in \{0,1\}} (3 \times \tilde{mult}_{2}((3,4), x_1,x_2,y_1,y_2) + 1 \times \tilde{mult}_{2}((1,2), x_1,x_2,y_1,y_2)) \tilde{V}_{2}(x_1,x_2) \tilde{V}_{2}(y_1,y_2)
$$

we can further rewrite this equation to be 

$$
\sum_{x_1,x_2,y_1,y_2 \in \{0,1\}} \tilde{Mult}_{2}(x_1,x_2,y_1,y_2) \tilde{V}_{2}(x_1,x_2) \tilde{V}_{2}(y_1,y_2)
$$

where

$$
\begin{aligned}
\tilde{Mult}_{2}(x_1,x_2,y_1,y_2) = (3 \times \tilde{mult}_{2}((3,4), x_1,x_2,y_1,y_2) + 1 \times \tilde{mult}_{2}((1,2), x_1,x_2,y_1,y_2))
\end{aligned}
$$

So the proving target becomes 

$$
1 =?= \sum_{x_1,x_2,y_1,y_2 \in \{0,1\}} \tilde{Mult}_{2}(x_1,x_2,y_1,y_2) \tilde{V}_{2}(x_1,x_2) \tilde{V}_{2}(y_1,y_2)
$$

#### Phase 1

Unlike the regular sumcheck in Step 2, here we introduce a lookup table $h_2(x_1,x_2)$ to speedup the process of sumcheck (You can still use the regular sumcheck protocol)

Before we do that, we need to calculate some MLE expression for $\tilde{Mult}_{2}(x_1,x_2,y_1,y_2)$ and $\tilde{V}_{2}(b_1,b_2)$

Audiences can do the calculation by themselves to double check if they match.

My calculation is shown as below:

$$
\tilde{V}_{2}(b_1,b_2) = 2(1-b_1)(1-b_2) + (1-b_1)b_2 + b_1(1-b_2) + 4b_1b_2 = 2 - b_1 - b_2 + 4b_1b_2
$$

$$
\begin{aligned}
3 \times \tilde{mult}_{2}((3,4), x_1,x_2,y_1,y_2) =& 3 \times ( (1-3)(1-4)(1-x_1)(1-x_2)(1-y_1)(y2)+\\
&(1-3)4x_1(1-x_2)y_1y_2+\\
&3(1-4)(1-x_1)x_2y_1(1-y_2)+\\
&3\times 4 (1-x_1)(1-x_2)y_1y_2 ) \\
=&3(1-x_1)(1-x_2)(1-y_1)(y2)+\\
&x_1(1-x_2)y_1y_2+\\
&3(1-x_1)x_2y_1(1-y_2)+\\
&(1-x_1)(1-x_2)y_1y_2 ) \\
\end{aligned}
$$

$$
\begin{aligned}
1 \times \tilde{mult}_{2}((1,2), x_1,x_2,y_1,y_2) =& ( (1-1)(1-1)(1-x_1)(1-x_2)(1-y_1)(y2)+\\
&(1-1)x_1(1-x_2)y_1y_2+\\
&(1-1)(1-x_1)x_2y_1(1-y_2)+\\
&1\times 1 (1-x_1)(1-x_2)y_1y_2 ) \\
=&(1-x_1)(1-x_2)y_1y_2
\end{aligned}
$$

So we have

$$
\begin{aligned}
\tilde{Mult}_{2}(x_1,x_2,y_1,y_2) =& \\
&3(1-x_1)(1-x_2)(1-y_1)(y2)+\\
&x_1(1-x_2)y_1y_2+\\
&3(1-x_1)x_2y_1(1-y_2)+\\
&2(1-x_1)(1-x_2)y_1y_2
\end{aligned}
$$

And in order to calculate the $h_2(x_1,x_2)$, we have

$$
h_2(x_1,x_2) = \sum_{y_1,y_2 \in \{0,1\}} \tilde{Mult}_{2}(x_1,x_2,y_1,y_2) \tilde{V}_{2}(y_1,y_2)
$$

Because travsing the all combination of $(y_1, y_2)$ is like travsing the binary tree, where if $y_i$ is 0, we calculate $(1-y_i)$ in MLE and where $y_i$ is 1, we have $y_i$. By doing so, we can construct the lookup table $h_2(x_1,x_2)$ in O(n) time as

$$
h_2(x_1,x_2) = (1-x_1)(1-x_2) + 3(1-x_1)x_2
$$

So the proving target becomes

$$
\begin{aligned}
1 =?=& \sum_{x_1,x_2 \in \{0,1\}} h_2(x_1,x_2) \tilde{V}_{2}(x_1,x_2) \\
=& \sum_{x_1,x_2 \in \{0,1\}} ((1-x_1)(1-x_2) + 3(1-x_1)x_2) (2 - x_1 - x_2 + 4x_1x_2)
\end{aligned}
$$

And then rest of computation becomes the same as Step 2 (regular sumcheck), which we will skip here. At the end of Phase 1, we will have a random number for $(x_1,x_2)$, let say $(x_1,x_2) = (4,3)$

In [62]:
# Like the regular GKR, prover will need to know MLE for V2
V2_MLE = generate_multilinear_extension([2, 1, 1, 4])
print(f"V2_MLE(0, 0) = {V2_MLE([0, 0])})")
print(f"V2_MLE(0, 1) = {V2_MLE([0, 1])})")
print(f"V2_MLE(1, 0) = {V2_MLE([1, 0])})")
print(f"V2_MLE(1, 1) = {V2_MLE([1, 1])})")

# Because there are only mult. gates between layer 1 and layer 2
# mult2_tilde([0,0,0,0,0,1]) = 1 --> 6b'100000 = 32
# mult2_tilde([0,1,1,0,1,1]) = 1 --> 6b'110110 = 54
# mult2_tilde([1,0,0,1,1,0]) = 1 --> 6b'011001 = 25
# mult2_tilde([1,1,0,0,1,1]) = 1 --> 6b'110011 = 51
mult2_tilde_values = [0] * 64
mult2_tilde_values[32] = 1
mult2_tilde_values[54] = 1
mult2_tilde_values[25] = 1
mult2_tilde_values[51] = 1

print("Build multilinear extension for mult2_tilde")
mult2_tilde_mle_raw = generate_multilinear_extension(mult2_tilde_values)
print(f"mult2_tilde_mle_raw([0,0,0,0,0,1]) = {mult2_tilde_mle_raw([0,0,0,0,0,1])}")
print(f"mult2_tilde_mle_raw([0,1,1,0,1,1]) = {mult2_tilde_mle_raw([0,1,1,0,1,1])}")
print(f"mult2_tilde_mle_raw([1,0,0,1,1,0]) = {mult2_tilde_mle_raw([1,0,0,1,1,0])}")
print(f"mult2_tilde_mle_raw([1,1,0,0,1,1]) = {mult2_tilde_mle_raw([1,1,0,0,1,1])}")

print("Filling with random challenge from layer 1 (terms belongs to alpha)")
mult2_tilde_alpha = reduce_variables(mult2_tilde_mle_raw, [random_x1, random_x2, 'x1', 'x2', 'y1', 'y2'])
print(f"mult2_tilde_alpha([0,0,0,1]) = {mult2_tilde_alpha([0, 0, 0, 1])}")
print(f"mult2_tilde_alpha([1,0,1,1]) = {mult2_tilde_alpha([1, 0, 1, 1])}")
print(f"mult2_tilde_alpha([0,1,1,0]) = {mult2_tilde_alpha([0, 1, 1, 0])}")
print(f"mult2_tilde_alpha([0,0,1,1]) = {mult2_tilde_alpha([0, 0, 1, 1])}")

print("Filling with random challenge from layer 1 (terms belongs to beta)")
mult2_tilde_beta = reduce_variables(mult2_tilde_mle_raw, [random_y1, random_y2, 'x1', 'x2', 'y1', 'y2'])
print(f"mult2_tilde_beta([0,0,0,1]) = {mult2_tilde_beta([0, 0, 0, 1])}")
print(f"mult2_tilde_beta([1,0,1,1]) = {mult2_tilde_beta([1, 0, 1, 1])}")
print(f"mult2_tilde_beta([0,1,1,0]) = {mult2_tilde_beta([0, 1, 1, 0])}")
print(f"mult2_tilde_beta([0,0,1,1]) = {mult2_tilde_beta([0, 0, 1, 1])}")

# We can still use the "trivial" way to compute GKR for the layer 2, by building the same function like `V0_g` above, we can build V1_g:
# Note: we need to include random combination of alpha and beta
def V1_g_trivial(input_vars):
    x1, x2, y1, y2 = input_vars
    alpha_term = alpha * mult2_tilde_alpha([x1, x2, y1, y2]) * V2_MLE([x1, x2]) * V2_MLE([y1, y2])
    beta_term  = beta  * mult2_tilde_beta([x1, x2, y1, y2]) * V2_MLE([x1, x2]) * V2_MLE([y1, y2])
    print(f"x1 = {x1}, x2 = {x2}, y1 = {y1}, y2 = {y2},  alpha_term = {alpha_term}, beta_term = {beta_term}")
    return (alpha_term + beta_term) % modulus

# We can further build a hg function that traverse all y1, y2 combination
# The code beloe is only for demonstration purpose, in actual implementation, it will be done via lookup table
# the key step of traverse (the `eq` step) is shown later
def hg(xs):
    x1, x2 = xs
    term00 = alpha * mult2_tilde_alpha([x1, x2, 0, 0]) * V2_MLE([0, 0]) + beta * mult2_tilde_beta([x1, x2, 0, 0]) * V2_MLE([0, 0])
    term01 = alpha * mult2_tilde_alpha([x1, x2, 0, 1]) * V2_MLE([0, 1]) + beta * mult2_tilde_beta([x1, x2, 0, 1]) * V2_MLE([0, 1])
    term10 = alpha * mult2_tilde_alpha([x1, x2, 1, 0]) * V2_MLE([1, 0]) + beta * mult2_tilde_beta([x1, x2, 1, 0]) * V2_MLE([1, 0])
    term11 = alpha * mult2_tilde_alpha([x1, x2, 1, 1]) * V2_MLE([1, 1]) + beta * mult2_tilde_beta([x1, x2, 1, 1]) * V2_MLE([1, 1])
    return term00 + term01 + term10 + term11
    
def V1_hg_linear(xs):
    x1, x2 = xs
    return (V2_MLE([x1, x2]) * hg(xs)) % modulus

def H1_hg_linear_x1(x1):
    return V1_hg_linear([x1, 0]) + V1_hg_linear([x1, 1])

H1_hg_linear_0 = H1_hg_linear_x1(0)
H1_hg_linear_1 = H1_hg_linear_x1(1)
print(f"Verifier; H1_hg_linear_x1(0) = {H1_hg_linear_0}, H1_hg_linear_x1(1) = {H1_hg_linear_1}, Pass = {(H1_hg_linear_0 + H1_hg_linear_1)%modulus == ProveTarget}")

V2_MLE(0, 0) = 2)
V2_MLE(0, 1) = 1)
V2_MLE(1, 0) = 1)
V2_MLE(1, 1) = 4)
Build multilinear extension for mult2_tilde
mult2_tilde_mle_raw([0,0,0,0,0,1]) = 1
mult2_tilde_mle_raw([0,1,1,0,1,1]) = 1
mult2_tilde_mle_raw([1,0,0,1,1,0]) = 1
mult2_tilde_mle_raw([1,1,0,0,1,1]) = 1
Filling with random challenge from layer 1 (terms belongs to alpha)
mult2_tilde_alpha([0,0,0,1]) = 4
mult2_tilde_alpha([1,0,1,1]) = 2
mult2_tilde_alpha([0,1,1,0]) = 0
mult2_tilde_alpha([0,0,1,1]) = 0
Filling with random challenge from layer 1 (terms belongs to beta)
mult2_tilde_beta([0,0,0,1]) = 0
mult2_tilde_beta([1,0,1,1]) = 0
mult2_tilde_beta([0,1,1,0]) = 4
mult2_tilde_beta([0,0,1,1]) = 2
Verifier; H1_hg_linear_x1(0) = 2, H1_hg_linear_x1(1) = 2, Pass = True


In [63]:
random_v2_x1 = random.randrange(modulus)
H2_hg_linear_random_v2_x1 = H1_hg_linear_x1(random_v2_x1)
print(f"Verifier returns random_v2_x1 = {random_v2_x1}, H1_hg_linear_x1(random_v2_x1) = {H2_hg_linear_random_v2_x1}")

Verifier returns random_v2_x1 = 3, H1_hg_linear_x1(random_v2_x1) = 1


In [64]:
## We can proceed to next variable x2, same as regular GKR
def H2_hg_linear_x2(x2):
    return V1_hg_linear([random_v2_x1, x2])

H2_hg_linear_0 = H2_hg_linear_x2(0)
H2_hg_linear_1 = H2_hg_linear_x2(1)
print(f"Verifier; H2_hg_linear_x2(0) = {H2_hg_linear_0}, H2_hg_linear_x2(1) = {H2_hg_linear_1}, Pass = {(H2_hg_linear_0 + H2_hg_linear_1)%modulus == H1_hg_linear_x1(random_v2_x1)}")
random_v2_x2 = random.randrange(modulus)
H2_hg_linear_random_v2_x2 = H2_hg_linear_x2(random_v2_x2)

Verifier; H2_hg_linear_x2(0) = 1, H2_hg_linear_x2(1) = 0, Pass = True


From the procedure above, we can clearly feel that the computation needed is much less than regular GKR procedure.

The trick is that we can combine the mult2\_tilde\_alpha([x1, x2, y1, y2]) * V2\_MLE([y1, y2])
So that we can build a function hg_alpha(x1, x2), by traverse all combination of y1, y2

Hfull(x1, x2, y1, y2)

= alpha  * mult2\_tilde\_alpha([x1, x2, y1, y2]) * V2\_MLE([y1, y2]) + beta  * mult2\_tilde\_beta([x1, x2, y1, y2]) *  V2\_MLE([y1, y2])

the trivial way is the we calculate all (y1, y2) as
Hfull(x1, x2, 0, 0), Hfull(x1, x2, 0, 1), Hfull(x1, x2, 1, 0), Hfull(x1, x2, 1, 1)
We can notice the nature of computing MLE is that:

*When you meet 1, you times the current position factor r; if you have 0, you times (1-r)*. 

The code section is exactly the way we used to compute MLE evaluation for previous example.

We can call this "traverse-all-combination" function to be `eq`, optimize it to be linear time:

In [65]:
print("""
result = 0
for key, value in table.items():
    prod = value
    for i in range(n):
        if key[i] == 1:
            prod *= input_vars[i]
        else:
            prod *= (1 - input_vars[i])
    result += prod
""")


result = 0
for key, value in table.items():
    prod = value
    for i in range(n):
        if key[i] == 1:
            prod *= input_vars[i]
        else:
            prod *= (1 - input_vars[i])
    result += prod



The original method we used above is straight-forward, where we `*= input_vars[i]` if `key[i]` is 1, and `*= (1 - input_vars[i])` if it is not. We can write the function below:

In [66]:
mod = 10 ** 9 + 7
def eq(r, b):
    res = 1
    for i in range(len(r)):
        if ((b >> i) & 1) == 0:
            res *= (1 - r[i])
        else:
            res *= r[i]
        res %= mod
    return res

r = [i + 1 for i in range(20)]
eq_binary = [1 for i in range(2 ** len(r))]

The eq function is a core component of our optimization strategy. Here's what it does:

1. It efficiently computes the evaluation of our multilinear extension at a given point.
2. Instead of calculating this for every possible input combination, we use a binary tree approach.
3. This allows us to compute all possible evaluations in linear time, greatly speeding up our protocol.

The subsequent code blocks demonstrate different optimization techniques building on this idea.


In our case, the `b` stands for the (y1, y2) bit string, which has four combination (00,01,10,11). But this is too small to show the difference, so we changed it to 20 variables, to show the speedup.

For every bit we meet, we can build a binary tree, where going to left child means `*= input_vars[i]` and going right means `*= (1 - input_vars[i])`, so we have

In [67]:
def dfs(index, depth, answer):
    if depth == len(r):
        eq_binary[index] = answer
    else:
        dfs(index | (1 << depth), depth + 1, answer * r[depth] % mod)
        dfs(index, depth + 1, answer * (1 - r[depth]) % mod)

There is another optimization technique. Becuase the multiplication is combinational, we can split the bit string into two halfs and do a cross-product at the end

In [68]:
import time
# simple opt, splitting into two half and doing cross-product later
t0 = time.time()
lgn = len(r)
eq_bf_0 = [eq(r[:lgn//2], b) for b in range(2 ** (lgn // 2))]         # sqrt(n) * log(n)
eq_bf_1 = [eq(r[lgn//2:], b) for b in range(2 ** (lgn - (lgn // 2)))] # sqrt(n) * log(n)
eq_opt  = [x * y % mod for y in eq_bf_1 for x in eq_bf_0] # n
t1 = time.time()

We can now compare the difference in terms of speedup:

In [69]:
print(f"Time of `spliting-cross-product` = {t1 - t0}")

# brute force
t2 = time.time()
eq_bf = [eq(r, b) for b in range(2**len(r))]
t3 = time.time()
print(f"Time of `brute-force` = {t3 - t2}")

# DFS
t4 = time.time()
dfs(0, 0, 1)
t5 = time.time()
print(f"Time of `DFS` = {t5 - t4}")

Time of `spliting-cross-product` = 0.048887014389038086
Time of `brute-force` = 2.347182035446167
Time of `DFS` = 0.3650224208831787


We can clearly see that using `spliting-cross-product` is the fastest way to compute all combination of bits. In our case, we compute the temporary factor for y0 = 0 and 1. Same approach can be applied to y1 = 0 and 1. By doing cross-product, we can get all combinations needed to build $h_2(x_1, x_2)$ and the rest of computation is same as regular GKR, but number of variables is reduced by half.

#### Phase 2

Similar to Phase 1, we also need to calculate the lookup table for $(y_1, y_2)$ by replace $(x_1, x_2)$ with the random number that *Verifier* picked in Phase 1.

$$
h_2(y_1,y_2) = 3(1-y_1)y_2 + 2y_1y_2+3y_1(1-y_2)+2y_1y_2
$$

Then the proving target become 

$$
\sum_{y_1,y_2 \in \{0,1\}} h_2(y_1,y_2) \tilde{V}_{2}(y_1,y_2) = \sum_{y_1,y_2 \in \{0,1\}} (3(1-y_1)y_2 + 2y_1y_2+3y_1(1-y_2)+2y_1y_2) (2 - x_1 - x_2 + 4x_1x_2)
$$

which also follows the regular sumcheck protocol.

In [70]:
def hg4y(ys):
    y1, y2 = ys
    term = alpha * mult2_tilde_alpha([random_v2_x1, random_v2_x2, y1, y2]) * V2_MLE([y1, y2]) + beta * mult2_tilde_beta([random_v2_x1, random_v2_x2, y1, y2]) * V2_MLE([y1, y2])
    return term % modulus

def V1_hg4y_linear(ys):
    y1, y2 = ys
    return (V2_MLE([y1, y2]) * hg4y(ys)) % modulus

def H3_hg4y_linear_y1(y1):
    return V1_hg4y_linear([y1, 0]) + V1_hg4y_linear([y1, 1])

H3_hg4y_linear_0 = H3_hg4y_linear_y1(0)
H3_hg4y_linear_1 = H3_hg4y_linear_y1(1)
print(f"Verifier; H3_hg4y_linear_y1(0) = {H3_hg4y_linear_0}, H3_hg4y_linear_y1(1) = {H3_hg4y_linear_1}, Pass = {(H3_hg4y_linear_0 + H3_hg4y_linear_1)%modulus == H2_hg_linear_random_v2_x2}")
random_v2_y1 = random.randrange(modulus)
H3_hg_linear_random_v2_y1 = H3_hg4y_linear_y1(random_v2_y1)

Verifier; H3_hg4y_linear_y1(0) = 1, H3_hg4y_linear_y1(1) = 3, Pass = True



At the end of Linear GKR, *Verifier* will have two claims $V_3(r^{x_1}, r^{x_2})$ and $V_3(r^{y_1}, r^{y_2})$, where $r^{x_1}, r^{x_2}, r^{y_1}, r^{y_2}$ are the random challenges. *Verifier* will open these two claims via PCS (like KZG commitment) to produce the final accept or reject of this proof.

## Conclusion

This tutorial illustrates how to compute GKR proof by hand and using two lookup table to reduce the proof generation time complexicity to be linear.

In [71]:
## We reach the end of our proving procedure, we need to calculate the final H4
def H4_hg4y_linear_y2(y2):
    return V1_hg4y_linear([random_v2_y1, y2])

H4_hg4y_linear_0 = H4_hg4y_linear_y2(0)
H4_hg4y_linear_1 = H4_hg4y_linear_y2(1)
print(f"Verifier; H4_hg4y_linear_y2(0) = {H4_hg4y_linear_0}, H4_hg4y_linear_y2(1) = {H4_hg4y_linear_1}, Pass = {(H4_hg4y_linear_0 + H4_hg4y_linear_1)%modulus == H3_hg_linear_random_v2_y1}")
random_v2_y2 = random.randrange(modulus)
H4_hg_linear_random_v2_y2 = H4_hg4y_linear_y2(random_v2_y2)

print(f"[{random_v2_x1}, {random_v2_x2}, {random_v2_y1}, {random_v2_y2}] is the random points we chosen to proceed to next layer. If layer 2 is the input layer, we can proceed to do PCS")

Verifier; H4_hg4y_linear_y2(0) = 0, H4_hg4y_linear_y2(1) = 1, Pass = True
[3, 4, 0, 2] is the random points we chosen to proceed to next layer. If layer 2 is the input layer, we can proceed to do PCS


Developed by [Simon Lau](mailto:sl@polyhedra.network) from Polyhedra Network
© 2024 Polyhedra Network. All rights reserved.